In [1]:
import xarray as xr
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings


In [2]:
path='/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Volumetric Soil Moisture/*.nc'
soil_moisture=xr.open_mfdataset(path)

In [3]:
soil_moisture

<xarray.Dataset>
Dimensions:    (lat: 94, lon: 192, time: 15340, nbnds: 2)
Coordinates:
  * lat        (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
  * lon        (lon) float32 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Dimensions without coordinates: nbnds
Data variables:
    soilw      (time, lat, lon) float32 dask.array<chunksize=(365, 94, 192), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/03/13 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In the context of this dataset, the unit "fraction" is used to express volumetric soil moisture. Volumetric soil moisture represents the amount of water present in the soil as a fraction or percentage of the total volume of the soil.

Here's a more detailed explanation:

- **Volumetric Soil Moisture**: Soil moisture is the amount of water contained in the soil. Volumetric soil moisture specifically expresses this as a fraction of the total volume of the soil.

- **Unit "Fraction"**: In this dataset, soil moisture is expressed as a fraction, which means it is a ratio of the volume of water to the total volume of the soil. It is typically expressed as a value between 0 and 1, where:
  - 0 represents completely dry soil (no water content).
  - 1 represents completely saturated soil (all available pore spaces filled with water).
  - Values between 0 and 1 represent varying degrees of soil moisture content. For example, 0.5 would mean that half of the soil's pore spaces are filled with water.

In summary, when you see "fraction" as the unit for soil moisture, it means that the dataset is providing information about the proportion of the soil's volume that is occupied by water. This is a common unit for expressing soil moisture in scientific and environmental datasets.

#### we need to fix the grid size.

In [4]:
new_lat = np.arange(90, -91, -02.5).astype(np.float32)
new_lon = np.arange(0, 360, 02.5).astype(np.float32)

# Resample to the new grid size
resized_soil_moisture = soil_moisture.interp(lat=new_lat, lon=new_lon)

In [5]:
lat_range = slice(29,18.0)  # take lat values in backwards directions.
lon_range = slice(85, 97)  # 88.0 degrees east to 92.7 degrees east

# Use the sel method to select data within the specified Bangladesh region
bangladesh_resized_soil_moisture = resized_soil_moisture.sel(lat=lat_range,lon=lon_range)
bangladesh_resized_soil_moisture

<xarray.Dataset>
Dimensions:    (time: 15340, lat: 4, lon: 5, nbnds: 2)
Coordinates:
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
  * lat        (lat) float32 27.5 25.0 22.5 20.0
  * lon        (lon) float32 85.0 87.5 90.0 92.5 95.0
Dimensions without coordinates: nbnds
Data variables:
    soilw      (time, lat, lon) float32 dask.array<chunksize=(365, 4, 5), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/03/13 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [7]:
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm# Import tqdm
warnings.filterwarnings('ignore')

# # Open your netCDF dataset
# ds = xr.open_dataset('your_dataset.nc')

# Define the chunk size
chunk_size = 500  # Adjust this based on your available memory

# Initialize an empty DataFrame to store the results
soilw_df = pd.DataFrame(columns=['time', 'lat', 'lon', 'soilw'])

# Calculate the number of chunks
num_chunks = len(bangladesh_resized_soil_moisture.time)

# Create a tqdm progress bar
for start in tqdm(range(0, num_chunks, chunk_size),desc='Main Loop'):
    end = start + chunk_size
    chunk = bangladesh_resized_soil_moisture.isel(time=slice(start, end))

    # Extract the data
    time = chunk.time.values
    lat = chunk.lat.values
    lon = chunk.lon.values
    soilw = chunk.soilw.values

    # Create a DataFrame for this chunk and append it to the main DataFrame
    chunk_df = pd.DataFrame({
        'time': time.repeat(len(lat) * len(lon)),
        'lat': np.tile(lat, len(time) * len(lon)),
        'lon': np.tile(lon, len(time) * len(lat)),
        'soilw': soilw.ravel()
    })

    soilw_df = soilw_df.append(chunk_df, ignore_index=True)

# t2m_df now contains the concatenated data
soilw_df

Main Loop:   0%|          | 0/31 [00:00<?, ?it/s]

,time,lat,lon,soilw
0,1981-01-01,27.5,85.0,0.256957
1,1981-01-01,25.0,87.5,0.272241
2,1981-01-01,22.5,90.0,0.272919
3,1981-01-01,20.0,92.5,0.263421
4,1981-01-01,27.5,95.0,0.317205
...,...,...,...,...
306795,2022-12-31,20.0,85.0,NaN
306796,2022-12-31,27.5,87.5,NaN
306797,2022-12-31,25.0,90.0,NaN
306798,2022-12-31,22.5,92.5,NaN
